# Pipeline for ORCA-CLEAN Deep Denoizing Network

**Requirements** - In order to run this notebook, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [20]:
import datetime

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import AmlCompute, Environment, Model, Data
from azure.ai.ml.constants import AssetTypes

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [21]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [22]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

compute_instance_name = "whalesdenoisinggpu2"
try:
    print(ml_client.compute.get(compute_instance_name))
except Exception as ex:
    compute_instance = AmlCompute(
        name=compute_instance_name,
        type="amlcompute",
        size="Standard_NC24ads_A100_v4",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )
    ml_client.begin_create_or_update(compute_instance).result()
    print(f"Created new compute instance: {ml_client.compute.get(compute_instance_name)}")

Found the config file in: /config.json


enable_node_public_ip: true
id: /subscriptions/03fd01f6-6051-4545-a78e-ceaace399b96/resourceGroups/lianatests/providers/Microsoft.MachineLearningServices/workspaces/humpbackwhales-aml/computes/whalesdenoisinggpu2
idle_time_before_scale_down: 180
location: westeurope
max_instances: 4
min_instances: 0
name: whalesdenoisinggpu2
network_settings: {}
provisioning_state: Succeeded
size: STANDARD_NC24ADS_A100_V4
ssh_public_access_enabled: true
tier: dedicated
type: amlcompute



## 1.4 Create a custom environment

In [23]:
custom_env_name = "whales-denoising-env"
version = "1.8"
dependencies_dir = "./dependencies"

try:
    pipeline_job_env = ml_client.environments.get(name=custom_env_name, version=version)
    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )
except:
    pipeline_job_env = Environment(
        name=custom_env_name,
        description="Custom environment for running ORCA-CLEAN pipeline",
        conda_file=os.path.join(dependencies_dir, "conda.yaml"),
        image="mcr.microsoft.com/azureml/curated/acpt-pytorch-2.0-cuda11.7:17", #"mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04", #"mcr.microsoft.com/azureml/curated/acpt-pytorch-1.13-cuda11.7:18",
        version=version,
    )

    pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)
    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )

Environment with name whales-denoising-env is registered to workspace, the environment version is 1.8


## 1.5 Create the Data asset objects

In [24]:
whales_data_name = "whales_data"
noise_train_data_name = "noise_train_data"
noise_test_data_name = "noise_test_data"
noise_val_data_name = "noise_val_data"
version = "2"

In [25]:
path = "/mnt/humpbackwhales/data/denoising_data/whales_data_v2"
whales_data_name = "whales_data"

try:
    whales_data = ml_client.data.get(whales_data_name, version=version)
except:
    whales_data = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="whales vocalizations of all types: moans, barks, creaks, etc.",
        name=whales_data_name,
        version=version,
    )
    ml_client.data.create_or_update(whales_data)

In [26]:
path = "/mnt/humpbackwhales/data/denoising_data/noise_train_v2"
noise_train_data_name = "noise_train_data"

try:
    noise_train_data = ml_client.data.get(noise_train_data_name, version=version)
except:
    noise_train_data = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="5-sec segments of only-noise retrieved from raw hydrophone recordings",
        name=noise_train_data_name,
        version=version,
    )
    ml_client.data.create_or_update(noise_train_data)

In [27]:
path = "/mnt/humpbackwhales/data/denoising_data/noise_test_v2"
noise_test_data_name = "noise_test_data"

try:
    noise_test_data = ml_client.data.get(noise_test_data_name, version=version)
except:
    noise_test_data = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="5-sec segments of only-noise retrieved from raw hydrophone recordings",
        name=noise_test_data_name,
        version=version,
    )
    ml_client.data.create_or_update(noise_test_data)

In [28]:
path = "/mnt/humpbackwhales/data/denoising_data/noise_val_v2"
noise_val_data_name = "noise_val_data"

try:
    noise_val_data = ml_client.data.get(noise_val_data_name, version=version)
except:
    noise_val_data = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="5-sec segments of only-noise retrieved from raw hydrophone recordings",
        name=noise_val_data_name,
        version=version,
    )
    ml_client.data.create_or_update(noise_val_data)

In [29]:
path = "/mnt/humpbackwhales/denoising/outputs"
denoising_training_outputs_name = "denoising_training_outputs"

try:
    denoising_training_outputs = ml_client.data.get(denoising_training_outputs_name, version=version)
except:
    denoising_training_outputs = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="results of deep denoising network's training",
        name=denoising_training_outputs_name,
        version=version,
    )
    ml_client.data.create_or_update(denoising_training_outputs)

In [30]:
default_datastore_name = "workspaceblobstore"  # workspaceblobstore, workspacefilestore

# 2. Define and create components into workspace
## 2.1 Load components from YAML

In [31]:
parent_dir = "."
train_model = load_component(source=parent_dir + "/train_model.yml")
#score_data = load_component(source=parent_dir + "/score_data.yml")
#eval_model = load_component(source=parent_dir + "/eval_model.yml")

## 2.2 Inspect loaded component

In [32]:
# Print the component as yaml
print(train_model)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
version: 0.0.1
display_name: Train Deep Denoising Model
description: The training component
type: command
inputs:
  whales_data:
    type: uri_folder
  noise_train_data:
    type: uri_folder
  noise_val_data:
    type: uri_folder
  noise_test_data:
    type: uri_folder
  max_train_epochs:
    type: integer
    default: '150'
  learning_rate:
    type: number
    default: '0.001'
  batch_size:
    type: integer
    default: '16'
  num_workers:
    type: integer
    default: '6'
  augmentation:
    type: integer
    default: '1'
  n_fft:
    type: integer
    default: '4096'
  hop_length:
    type: integer
    default: '441'
  sequence_len:
    type: integer
    default: '1280'
  freq_compression:
    type: string
    default: linear
outputs:
  model_dir:
    type: uri_folder
    mode: upload
  log_dir:
    type: uri_folder
    mode: upload
  checkpoint_dir:
    type: uri_folder
    mode: 

# 3. Pipeline job
## 3.1 Build pipeline

In [33]:
# Construct pipeline
@pipeline()
def pipeline_with_components_from_yaml(
    whales_data_input,
    noise_train_data_input,
    noise_val_data_input,
    noise_test_data_input,
    max_train_epochs,
    
):
    """Pipeline with components defined via yaml."""
    train_component = train_model(
        whales_data=whales_data_input,
        noise_train_data=noise_train_data_input,
        noise_val_data=noise_val_data_input,
        noise_test_data=noise_test_data_input,
        max_train_epochs=max_train_epochs
    )
    
    #mounted_output_dir = "azureml://datastores/workspaceblobstore/paths/azureml/denoising/ML_pipeline"
    
    train_component.outputs.model_dir = denoising_training_outputs.path + "/models/"
    train_component.outputs.log_dir = denoising_training_outputs.path + "/logs/"
    train_component.outputs.checkpoint_dir = denoising_training_outputs.path + "/model_checkpoints/"
    train_component.outputs.cache_dir = denoising_training_outputs.path + "/cache/"
    train_component.outputs.summary_dir = denoising_training_outputs.path + "/summary/"
    
    return {
        "trained_model": train_component.outputs.model_dir,
        "logs": train_component.outputs.log_dir,
        "checkpoints": train_component.outputs.checkpoint_dir,
        "cache": train_component.outputs.cache_dir,
        "summary": train_component.outputs.summary_dir
    }

whales_data_name = "whales_data"
noise_train_data_name = "noise_train_data"
noise_test_data_name = "noise_test_data"
noise_val_data_name = "noise_val_data"

pipeline_job = pipeline_with_components_from_yaml(
    whales_data_input=Input(type="uri_folder", path=whales_data.path),
    noise_train_data_input=Input(type="uri_folder", path=noise_train_data.path),
    noise_val_data_input=Input(type="uri_folder", path=noise_val_data.path),
    noise_test_data_input=Input(type="uri_folder", path=noise_test_data.path),
    max_train_epochs=2,
)

# set pipeline level compute
pipeline_job.settings.default_compute = compute_instance_name

In [34]:
# Inspect built pipeline
print(pipeline_job)

display_name: pipeline_with_components_from_yaml
description: Pipeline with components defined via yaml.
type: pipeline
inputs:
  whales_data_input:
    type: uri_folder
    path: azureml://subscriptions/03fd01f6-6051-4545-a78e-ceaace399b96/resourcegroups/lianatests/workspaces/humpbackwhales-aml/datastores/workspaceblobstore/paths/LocalUpload/c4f8edf663ec11fe0d34c4808ef63531/whales_data_v2/
  noise_train_data_input:
    type: uri_folder
    path: azureml://subscriptions/03fd01f6-6051-4545-a78e-ceaace399b96/resourcegroups/lianatests/workspaces/humpbackwhales-aml/datastores/workspaceblobstore/paths/LocalUpload/2a9e34de5512ddd07b23092d3f2dd598/noise_train_v2/
  noise_val_data_input:
    type: uri_folder
    path: azureml://subscriptions/03fd01f6-6051-4545-a78e-ceaace399b96/resourcegroups/lianatests/workspaces/humpbackwhales-aml/datastores/workspaceblobstore/paths/LocalUpload/98989997927c524eb723a11cea8afec0/noise_val_v2/
  noise_test_data_input:
    type: uri_folder
    path: azureml://su

## 3.2 Submit pipeline job

In [35]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="whales_denoising"
)
pipeline_job

Uploading orca_clean (0.26 MBs): 100%|██████████| 262803/262803 [00:00<00:00, 374134.71it/s]




Experiment,Name,Type,Status,Details Page
whales_denoising,teal_actor_t1g8n1rkbc,pipeline,Preparing,Link to Azure Machine Learning studio


In [36]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: teal_actor_t1g8n1rkbc
Web View: https://ml.azure.com/runs/teal_actor_t1g8n1rkbc?wsid=/subscriptions/03fd01f6-6051-4545-a78e-ceaace399b96/resourcegroups/lianatests/workspaces/humpbackwhales-aml

Streaming logs/azureml/executionlogs.txt

[2023-09-16 21:21:01Z] Submitting 1 runs, first five are: 985f7350:900055b4-ad0b-4f99-9750-be15503d98a4
[2023-09-16 21:33:25Z] Completing processing run id 900055b4-ad0b-4f99-9750-be15503d98a4.

Execution Summary
RunId: teal_actor_t1g8n1rkbc
Web View: https://ml.azure.com/runs/teal_actor_t1g8n1rkbc?wsid=/subscriptions/03fd01f6-6051-4545-a78e-ceaace399b96/resourcegroups/lianatests/workspaces/humpbackwhales-aml



## 3.3 Register a model

In [ ]:
run_model = Model(
    path="azureml://jobs/{}/outputs/artifacts/paths/model/".format(pipeline_job.name),
    name="orca-clean-model-v1",
    description="Model trained with 5 epochs, whales_data_v2 and noise_train/test/val_v2.",
    type=AssetTypes.MLFLOW_MODEL
)

ml_client.models.create_or_update(run_model)

## 3.4 Create a batch endpoint

In [ ]:
from azure.ai.ml.entities import BatchEndpoint, ModelBatchDeployment, PipelineComponentBatchDeployment

endpoint_name="whales-denoising-batch"

endpoint = BatchEndpoint(
    name=endpoint_name,
    description="Endpoint for pipeline deployments",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

endpoint = ml_client.batch_endpoints.get(name=endpoint_name)
print(endpoint)

## 3.5 Deploy the pipeline job

In [ ]:
deployment = PipelineComponentBatchDeployment(
    name="whales-denoising-batch-from-job",
    description="This deployment is created from a pipeline job.",
    endpoint_name=endpoint.name,
    job_definition=pipeline_job,
    settings={
        "default_compute": compute_instance_name,
        "continue_on_step_failure": False
    }
)

In [ ]:
ml_client.batch_deployments.begin_create_or_update(deployment).result()

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint.name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

## 3.6 Test the deployment

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
)

In [ ]:
ml_client.jobs.get(name=job.name)